# Language Modeling

In [32]:
import spacy
import nltk
from nltk.util import ngrams
from sklearn.model_selection import train_test_split
from collections import defaultdict, Counter

In [49]:
#Donald Trump Speeches: https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt
with open("speeches.txt", 'r', encoding='utf8', errors='ignore') as f:
    text = f.readlines()

speech_text = ""
for line in text:
    if line.strip() and not (line.strip().startswith("SPEECH ")):
        speech_text += " " + line.strip()

sentences = nltk.sent_tokenize(speech_text)
corpus = []
for sent in sentences:
    sent = sent.replace("...", " ")
    sent = sent.replace("--", " ")
    sent = sent.replace("’", "'")
    sent = sent.replace("—", "-")
    sent = sent.replace("``", "'")
    if sent.isupper():
        sent = sent[0] + sent[1:].lower()
    corpus.append(sent)

In [50]:
vocab = defaultdict()
processed_corpus = []

def preprocess_and_construct_vocab(corpus):
    for sent in corpus:
        tokens = nltk.word_tokenize(sent)
        lowercased_tokens = [x.lower() for x in tokens]
        sent_tokens = ["<s>"] + lowercased_tokens + ["</s>"]
        for tok in sent_tokens:
            if tok in vocab:
                vocab[tok] += 1
            else:
                vocab[tok] = 1
        processed_corpus.append(sent_tokens)
    return

In [51]:
preprocess_and_construct_vocab(corpus)

train, test = train_test_split(processed_corpus, test_size=0.2)

print("Train set size: ", len(train))
print("Test set size:", len(test))

Train set size:  13193
Test set size: 3299


## Classical Approach

In [55]:
def compute_ngrams(sent_tokens, n):
    return ngrams(sent_tokens, n)

def compute_all_possible_ngram_count(vocab_size, n):
    return vocab_size**n

In [62]:
bigrams = Counter()
trigrams = Counter()
quadgrams = Counter()

for sent_toks in train:
    bigrams.update([bg for bg in compute_ngrams(sent_toks, 2)])
    trigrams.update([tg for tg in compute_ngrams(sent_toks, 3)])
    quadgrams.update([qg for qg in compute_ngrams(sent_toks, 4)])


### Vocab Stats

In [77]:
print("Vocab size: ", len(vocab))

Vocab size:  6152


This is also the count of unique unigrams in the corpus.

### Bigram Stats

In [78]:
print("Overall bigram count: ", sum(bigrams.values()))
print("Count of unique bigrams: ", len(bigrams))
print("All possible bigrams in the corpus: ", compute_all_possible_ngram_count(len(vocab), 2))

Overall bigram count:  177055
Count of unique bigrams:  41665
All possible bigrams in the corpus:  37847104


### Trigram Stats

In [79]:
print("Overall trigram count: ", sum(trigrams.values()))
print("Count of unique trigrams: ", len(trigrams))
print("All possible trigrams in the corpus: ", compute_all_possible_ngram_count(len(vocab), 3))

Overall trigram count:  163862
Count of unique trigrams:  87090
All possible trigrams in the corpus:  232835383808


### Quadgram Stats

In [80]:
print("Overall quadgram count: ", sum(quadgrams.values()))
print("Count of unique quadgrams: ", len(quadgrams))
print("All possible trigrams in the corpus: ", compute_all_possible_ngram_count(len(vocab), 4))

Overall quadgram count:  150672
Count of unique quadgrams:  112493
All possible trigrams in the corpus:  1432403281186816


### Unigram Model

In unigram model, $P(w_n | w_{1}^{n-1}) = P(w_n)$.

Using Maximum Likelihood Estimation, $P(w_i) = \frac{\text{count}(w_i)}{\sum_{w} \text{count}(w)} = \frac{\text{count}(w_i)}{N}$.

Using Add One Smoothing, $P(w_i) = \frac{\text{count}(w_i) + 1}{N + V}$.

In [141]:
class UnigramModel:
    def __init__(self):
        return
    
    # Using MLE with Add One Smoothing
    def conditional_prob_of_tokens(self, unigr):
        unigr = unigr[0]
        return ((vocab[unigr] + 1)/(sum(vocab.values()) + len(vocab)))
    
    def perplexity_sent(self, sent_toks):
        perplexity = 1.0
        
        sent_unigrams = compute_ngrams(sent_toks, 1)
        for ug in sent_unigrams:
            perplexity = perplexity * (1/self.conditional_prob_of_tokens(ug))
            
        perplexity = perplexity ** (1/len(sent_toks))
        if perplexity == math.inf:
            #print(sent_toks)
            perplexity = 0.0
        return perplexity

### Bigram Model

Using Maximum Likelihood Estimation, $P(w_i | w_{i-1}) = \frac{\text{count}(w_{i-1}, w_i)}{\sum_{w} \text{count}(w_{i-1}, w)} = \frac{\text{count}(w_{i-1}, w_i)}{ \text{count}(w_{i-1})}$

Using Add One Smoothing, $P(w_i | w_{i-1}) = \frac{\text{count}(w_{i-1}, w_i) + 1}{ \text{count}(w_{i-1}) + V}$

In [144]:
class BigramModel:
    
    def MLE_with_add_one_smoothing(self, w_i_minus_1, w_i):
        co_occurrence_count = bigrams[(w_i_minus_1, w_i)] + 1
        unigram_count = vocab[w_i_minus_1] + len(vocab)
        return co_occurrence_count/unigram_count
    
    def conditional_prob_of_tokens(self, bigram_tuple):
        return self.MLE_with_add_one_smoothing(bigram_tuple[0], bigram_tuple[1])
    
    def perplexity_sent(self, sent_toks):
        perplexity = 1.0
        
        sent_bigrams = compute_ngrams(sent_toks, 2)
        for bg in sent_bigrams:
            perplexity = perplexity * (1/self.conditional_prob_of_tokens(bg))
            
        perplexity = perplexity ** (1/len(sent_toks))
        if perplexity == math.inf:
            #print(sent_toks)
            perplexity = 0.0
        return perplexity

### Trigram Model

Using Maximum Likelihood Estimation and Add One Smoothing, $P(w_i | w_{i-2}, w_{i-1}) = \frac{\text{count}(w_{i-2}, w_{i-1}, w_i) + 1}{ \text{count}(w_{i-2}, w_{i-1}) + V}$

In [149]:
class TrigramModel:
    
    def MLE_with_add_one_smoothing(self, w_i_minus_2, w_i_minus_1, w_i):
        co_occurrence_count = trigrams[(w_i_minus_2, w_i_minus_1, w_i)] + 1
        bigram_count = bigrams[(w_i_minus_2, w_i_minus_1)] + len(vocab)
        return co_occurrence_count/bigram_count
    
    def conditional_prob_of_tokens(self, trigram_tuple):
        return self.MLE_with_add_one_smoothing(trigram_tuple[0], trigram_tuple[1], trigram_tuple[2])
    
    def perplexity_sent(self, sent_toks):
        perplexity = 1.0
        
        sent_bigrams = compute_ngrams(sent_toks, 3)
        for bg in sent_bigrams:
            perplexity = perplexity * (1/self.conditional_prob_of_tokens(bg))
            
        perplexity = perplexity ** (1/len(sent_toks))
        if perplexity == math.inf:
            #print(sent_toks)
            perplexity = 0.0
        return perplexity

### Quadgram Model

Using Maximum Likelihood Estimation and Add One Smoothing, $P(w_i | w_{i-3}, w_{i-2}, w_{i-1}) = \frac{\text{count}(w_{i-3}, w_{i-2}, w_{i-1}, w_i) + 1}{ \text{count}(w_{i-3}, w_{i-2}, w_{i-1}) + V}$

In [153]:
class QuadgramModel:
    
    def MLE_with_add_one_smoothing(self, w_i_minus_3, w_i_minus_2, w_i_minus_1, w_i):
        co_occurrence_count = quadgrams[(w_i_minus_3, w_i_minus_2, w_i_minus_1, w_i)] + 1
        trigram_count = trigrams[(w_i_minus_3, w_i_minus_2, w_i_minus_1)] + len(vocab)
        return co_occurrence_count/trigram_count
    
    def conditional_prob_of_tokens(self, quadgram_tuple):
        return self.MLE_with_add_one_smoothing(quadgram_tuple[0], quadgram_tuple[1], quadgram_tuple[2], quadgram_tuple[2])
    
    def perplexity_sent(self, sent_toks):
        perplexity = 1.0
        
        sent_bigrams = compute_ngrams(sent_toks, 4)
        for bg in sent_bigrams:
            perplexity = perplexity * (1/self.conditional_prob_of_tokens(bg))
            
        perplexity = perplexity ** (1/len(sent_toks))
        if perplexity == math.inf:
            #print(sent_toks)
            perplexity = 0.0
        return perplexity

### Evaluation: Perplexity on Test Set

In [156]:
unigram_model = UnigramModel()
bigram_model = BigramModel()
trigram_model = TrigramModel()
quadgram_model = QuadgramModel()

uni_test_pps = []
bi_test_pps = []
tri_test_pps = []
quad_test_pps = []

for sent_toks in test:
    uni_test_pps.append(unigram_model.perplexity_sent(sent_toks))
    bi_test_pps.append(bigram_model.perplexity_sent(sent_toks))
    tri_test_pps.append(trigram_model.perplexity_sent(sent_toks))
    quad_test_pps.append(quadgram_model.perplexity_sent(sent_toks))
    
print("Average perplexity of the Unirgam model on the test dataset: ", np.mean(uni_test_pps))
print("Average perplexity of the Birgam model on the test dataset: ", np.mean(bi_test_pps))
print("Average perplexity of the Unirgam model on the test dataset: ", np.mean(tri_test_pps))
print("Average perplexity of the Birgam model on the test dataset: ", np.mean(quad_test_pps))

Average perplexity of the Unirgam model on the test dataset:  238.21660075056752
Average perplexity of the Birgam model on the test dataset:  226.22457207989984
Average perplexity of the Unirgam model on the test dataset:  590.0011143806028
Average perplexity of the Birgam model on the test dataset:  939.1964927310736


The unigram model is the naive model that calculates perplexity based on the frequency counts of individual words only, and bigram is a better model than the unigram model. The perplexity values for the trigram and the quadgram model are increasing as we are increasing the value of n, as our model gets more and more confused when it sees a longer stream of tokens that it hasn't seen before. We can improve these results by using advanced smoothing methods such as Good Turing with Interpolation.